---

<div style="text-align: center; font-family: Arial, sans-serif; margin-top: 50px;">
    <h1 style="font-size: 36px; font-weight: bold;"><b>Prácticas de NLP</b></h1>
    <h2 style="font-size: 28px; color: #2E86C1;"><b>NLP Basics Assessment  -  Clasificación de Sentimientos en Críticas de Películas</b></h2>
    <p style="font-size: 20px; margin-top: 30px;">
        <b>Materia:</b> Procesamiento de Lenguaje Natural<br>
        <b>Estudiantes:</b> Albin Rivera y Yesid Castelblanco<br>
        <b>Fecha:</b> 16 de Agosto de 2025
    </p>
</div>

---

# **Configuración del Entorno y Preparación de Librerías**
---

**Importación de librerías necesarias para las dos prácticas**

---

In [1]:
import warnings                     # Para manejar y suprimir mensajes de advertencia
import spacy                        # Librería de Procesamiento de Lenguaje Natural (NLP)
from spacy.matcher import Matcher   # Herramienta de SpaCy para definir y buscar patrones en el texto
import pkg_resources                # Para manejar recursos y dependencias de paquetes
import pandas as pd                 # Manipulación y análisis de datos en estructuras tipo DataFrame
import numpy as np                  # Operaciones numéricas y manejo de arreglos
import nltk                         # Librería para procesamiento de lenguaje natural
from nltk.sentiment.vader import SentimentIntensityAnalyzer  # Analizador de sentimiento VADER
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report     # Métricas de evaluación para modelos de clasificación
import pkg_resources                # Permite acceder a recursos dentro de paquetes Python, verificar versiones de librerías y manejar dependencias de manera programática.

/tmp/ipython-input-3125081599.py:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources                # Para manejar recursos y dependencias de paquetes


In [ ]:
warnings.filterwarnings('ignore')   # Desactiva las advertencias para mantener la salida más limpia

**Detección de entorno y configuración automática de dependencias en Google Colab**

---

<div align="justify">
Este bloque tiene como función principal detectar si el entorno de ejecución es Google Colab y, en caso afirmativo, instalar automáticamente las dependencias necesarias desde un archivo requirements.txt alojado en GitHub.
</div>

In [3]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

print("¿Ejecutando en Google Colab?:", IN_COLAB)

if IN_COLAB:
    !wget https://raw.githubusercontent.com/YesidCastelblanco/Fundamentos_NLP/main/requirements.txt -O requirements.txt
    !pip install -r requirements.txt
else:
    print("No estás en Google Colab. No se instalarán dependencias automáticamente.")


¿Ejecutando en Google Colab?: True
--2025-08-16 14:37:21--  https://raw.githubusercontent.com/YesidCastelblanco/Fundamentos_NLP/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 350 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     350  --.-KB/s    in 0s      

2025-08-16 14:37:21 (29.1 MB/s) - ‘requirements.txt’ saved [350/350]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/6

# **NLP Basics Assessment**

---

<div align="justify">
En este notebook se aplican técnicas de procesamiento de lenguaje natural a un corpus específico: <b>The Adventures of Sherlock Holmes</b> de Arthur Conan Doyle (1892). Esta obra es de dominio público y el corpus fue obtenido de <b>Project Gutenberg.</b>
</div>

**Creamos el documento desde el archivo sherlock_holmes.txt**

---

<div align="justify">
Este es un comando de shell que se ejecuta directamente desde el notebook gracias al prefijo !. En lugar de ser interpretado por Python, se envía al sistema operativo como si lo escribieramos en la terminal de Linux que corre detrás de Google Colab.
</div>

In [ ]:
!test '{IN_COLAB}' = 'True' && wget  https://raw.githubusercontent.com/YesidCastelblanco/Fundamentos_NLP/refs/heads/main/Unidad1/sherlock_holmes.txt

--2025-08-15 18:04:20--  https://raw.githubusercontent.com/YesidCastelblanco/Fundamentos_NLP/refs/heads/main/Unidad1/sherlock_holmes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607648 (593K) [text/plain]
Saving to: ‘sherlock_holmes.txt’

sherlock_holmes.txt 100%[===================>] 593.41K  --.-KB/s    in 0.02s   

2025-08-15 18:04:20 (24.5 MB/s) - ‘sherlock_holmes.txt’ saved [607648/607648]



**Cargar, limpiar y procesar el texto de Sherlock Holmes con SpaCy**

---

<div align="justify">
Este código carga el modelo de SpaCy en inglés (en_core_web_sm) y abre el archivo sherlock_holmes.txt para leer su contenido completo. Dado que los textos de Project Gutenberg incluyen encabezados y pies de página que no forman parte de la obra original, se emplean expresiones regulares para localizar las marcas *** START OF ... *** y *** END OF ... *** y así extraer únicamente el contenido entre ellas; en caso de no encontrarlas, se utiliza todo el texto. Una vez limpio, el corpus se procesa con SpaCy para generar un objeto doc, lo que permite aplicar técnicas de análisis lingüístico como tokenización, detección de oraciones o reconocimiento de entidades. Finalmente, se imprime la longitud del texto original frente al texto limpio, junto con una vista previa de los primeros 500 caracteres, lo que permite confirmar que la limpieza y carga del corpus se realizaron correctamente.
</div>

In [ ]:
# Cargar el modelo de SpaCy
nlp = spacy.load("en_core_web_sm")

# Abrir y leer el archivo
with open('sherlock_holmes.txt', 'r', encoding='utf-8') as file:
    texto = file.read()

# Eliminar encabezado y pie de página de Project Gutenberg
inicio = re.search(r"\*\*\* START OF(.*?)\*\*\*", texto, re.DOTALL)
fin = re.search(r"\*\*\* END OF(.*?)\*\*\*", texto, re.DOTALL)

if inicio and fin:
    texto_limpio = texto[inicio.end():fin.start()].strip()
else:
    texto_limpio = texto  # Si no encuentra marcas, usa todo el texto

# Procesar el texto limpio con Spacy
doc = nlp(texto_limpio)

# Información básica
print(f"Texto original: {len(texto)} caracteres")
print(f"Texto limpio: {len(texto_limpio)} caracteres\n")

# Mostrar primeros 500 caracteres del texto limpio
print("Primeros 500 caracteres del texto limpio:\n")
print(texto_limpio[:500])


Texto original: 581565 caracteres
Texto limpio: 562202 caracteres

Primeros 500 caracteres del texto limpio:

The Adventures of Sherlock Holmes

by Arthur Conan Doyle


Contents

   I.     A Scandal in Bohemia
   II.    The Red-Headed League
   III.   A Case of Identity
   IV.    The Boscombe Valley Mystery
   V.     The Five Orange Pips
   VI.    The Man with the Twisted Lip
   VII.   The Adventure of the Blue Carbuncle
   VIII.  The Adventure of the Speckled Band
   IX.    The Adventure of the Engineer’s Thumb
   X.     The Adventure of the Noble Bachelor
   XI.    The Adventure of the Beryl Coronet
 


**Visualizar los primeros 50 tokens del texto procesado**

---

In [ ]:
doc[:50]

The Adventures of Sherlock Holmes

by Arthur Conan Doyle


Contents

   I.     A Scandal in Bohemia
   II.    The Red-Headed League
   III.   A Case of Identity
   IV.    The Boscombe Valley Mystery
   V.     The Five Orange

**Cuantos tokens hay en el archivo?**

---

<div align="justify">
La instrucción len(doc) devuelve la cantidad total de tokens presentes en el objeto doc, es decir, el número de unidades lingüísticas que SpaCy identificó en el texto después de procesarlo. Cada token puede corresponder a una palabra, un signo de puntuación, un número u otros elementos del lenguaje. Este valor ofrece una primera idea de la extensión del corpus en términos de elementos analizados por el modelo de procesamiento de lenguaje natural.
</div>

In [ ]:
len(doc)

136993

**Cuantas oraciones hay en el archivo?**

---


<div align="justify">
La instrucción sentences = list(doc.sents) convierte en una lista todas las oraciones que SpaCy detecta en el texto procesado, aprovechando su capacidad de segmentación en frases. Posteriormente, len(sentences) devuelve el número total de oraciones identificadas en el corpus. Este resultado permite tener una medida de la extensión del texto no solo en tokens, sino también en unidades sintácticas más amplias, lo cual es útil para tareas de análisis lingüístico y de comprensión de la estructura narrativa.
</div>

In [ ]:
sentences = list(doc.sents)
len(sentences)

5800

**Imprime la segunda oración del documento**

---


<div align="justify">
La instrucción sentences[1] devuelve la segunda oración del texto procesado. Dado que en Python los índices comienzan en cero, sentences[0] corresponde a la primera oración, mientras que sentences[1] accede a la siguiente. Esta operación resulta útil para inspeccionar ejemplos concretos de cómo SpaCy ha segmentado el corpus y verificar que la detección de oraciones se esté realizando de manera adecuada.
</div>

In [ ]:
sentences[1]

The Red-Headed League
   III.   

**Por cada token en la oración anterior, imprime su `text`, `POS` tag, `dep` tag y `lemma`**
<br>

---

<div align="justify">
Imprime una tabla con información lingüística de cada token en la segunda oración del texto. En primer lugar, se define un encabezado con las columnas Text, POS, dep y lemma, que corresponden respectivamente al texto original del token, su categoría gramatical (Part of Speech), la relación de dependencia sintáctica dentro de la oración y la forma lematizada o base del término. Luego, mediante un bucle for, se recorren los tokens de sentences[1] y se imprime para cada uno el valor de estas propiedades. De esta manera, el resultado permite visualizar cómo SpaCy analiza los componentes gramaticales y semánticos de una oración en detalle, mostrando no solo las palabras, sino también su función sintáctica y su raíz léxica.
</div>

In [ ]:
print("{:20}{:20}{:20}{:20}".format("Text", "POS", "dep", "lemma"))
for token in sentences[1]:
    print(f"{token.text:{20}}{token.pos_:{20}}{token.dep_:{20}}{token.lemma_:{20}}")

Text                POS                 dep                 lemma               
The                 DET                 det                 the                 
Red                 PROPN               compound            Red                 
-                   PUNCT               punct               -                   
Headed              PROPN               compound            Headed              
League              PROPN               compound            League              

                   SPACE               dep                 
                   
III                 PROPN               ROOT                III                 
.                   PUNCT               punct               .                   
                    SPACE               dep                                     


**Definir patrón en SpaCy para detectar la frase "Baker Street"**

---


<div align="justify">
Matcher de SpaCy, una herramienta diseñada para buscar patrones específicos en un texto. Primero, se crea el objeto matcher = Matcher(nlp.vocab), que se apoya en el vocabulario del modelo cargado (nlp.vocab) para poder reconocer palabras y sus atributos. Después, se define un patrón pattern = [{'LOWER': 'baker'}, {'IS_SPACE': True}, {'LOWER': 'street'}]. Este patrón indica que se quiere identificar la secuencia exacta “baker street”, sin importar si está escrita en mayúsculas o minúsculas (gracias al atributo LOWER), y considerando que entre ambas palabras debe existir un espacio (IS_SPACE: True). Finalmente, con matcher.add("baker", [pattern]), se registra este patrón dentro del matcher bajo la etiqueta "baker". Así, posteriormente será posible buscar dentro del documento cualquier ocurrencia de “baker street” y obtener su posición en el texto.
</div>

In [ ]:
matcher = Matcher(nlp.vocab)
pattern = [{'LOWER': 'baker'}, {'IS_SPACE': True}, {'LOWER': 'street'}]
matcher.add("baker", [pattern])

**Ejecutar el matcher en el documento y obtener coincidencias**

---

<div align="justify">
Cuando se ejecuta found_matches = matcher(doc), el objeto matcher busca dentro del documento doc todas las coincidencias que correspondan al patrón previamente definido (en este caso, la secuencia “baker street”). El resultado se guarda en found_matches, que es una lista de tuplas. Cada tupla contiene tres elementos: el ID del patrón (internamente un número que corresponde al nombre “baker”), el índice inicial del fragmento en el documento, y el índice final. Estos índices permiten identificar con precisión la posición de la coincidencia dentro del texto. Al imprimir found_matches, no se verá directamente el texto encontrado, sino estos identificadores numéricos que luego se pueden usar para extraer los fragmentos coincidentes desde doc.
</div>

In [ ]:
found_matches = matcher(doc)
found_matches

[(9822559787564794947, 40299, 40302),
 (9822559787564794947, 66767, 66770),
 (9822559787564794947, 70527, 70530),
 (9822559787564794947, 74770, 74773)]

**Imprime el texto al rededor de cada match encontrado**

---

<div align="justify">
El código start, end = found_matches[0][1:] toma la primera coincidencia encontrada por el matcher (found_matches[0]) y extrae únicamente los índices de inicio y fin de esa coincidencia dentro del documento doc. Luego, con doc[start-9:end+13], se está ampliando artificialmente ese rango: en lugar de mostrar solo la coincidencia exacta (“baker street”), se retrocede 9 tokens antes de la coincidencia y se avanza 13 tokens después. El resultado es un fragmento más grande del texto que incluye la coincidencia en su contexto. Esto es útil porque permite no solo identificar dónde apareció el patrón, sino también leer la frase o pasaje alrededor de él para entender mejor el sentido en el que se usó.
</div>

In [ ]:
start, end = found_matches[0][1:]
doc[start-9:end+13]

had only known the quiet thinker and logician of Baker
Street would have failed to recognise him. His face flushed and
darkened

**Imprime la oración que contiene cada match encontrado**

---

<div align="justify">
Este código recorre todas las oraciones del documento (for sentence in sentences:) y, dentro de cada una, recorre las coincidencias encontradas por el matcher (for _, start, end in found_matches:). Luego, verifica si los índices de la coincidencia (start y end) caen dentro de los límites de esa oración (if sentence.start <= start and sentence.end >= end:). Si la coincidencia está efectivamente contenida dentro de la oración, se imprime el texto completo de la oración (print(sentence.text, '\n')).
En otras palabras, este fragmento permite encontrar la oración completa en la que apareció un patrón (por ejemplo, “baker street”), mostrando el contexto exacto de esa coincidencia.
</div>

In [ ]:
for sentence in sentences:
    for _, start, end in found_matches:
        if sentence.start <= start and sentence.end >= end:
            print(sentence.text, '\n')

Men who had only known the quiet thinker and logician of Baker
Street would have failed to recognise him. 

I think, Watson, that if we drive to Baker
Street we shall just be in time for breakfast.”




VII. 

Mr. Henry Baker
can have the same by applying at 6:30 this evening at 221B, Baker
Street.’ 

Then he stepped into
the cab, and in half an hour we were back in the sitting-room at Baker
Street. 



---

# **CONCLUSIONES**

---

<div align="justify">
<b>1.Procesamiento con spaCy:</b><br>
El uso de spaCy permitió dividir el texto en oraciones y aplicar el Matcher para identificar patrones específicos, como expresiones o nombres relevantes dentro del corpus.
<br><br>
<b>2.Extracción contextual:</b><br>
El código no solo detectó coincidencias, sino que también mostró la oración completa en la que aparecía cada patrón. Esto aporta un contexto semántico, mucho más útil que una simple búsqueda de palabras aisladas.
<br><br>
<b>3.Flexibilidad del Matcher:</b><br>
El Matcher de spaCy resultó una herramienta poderosa para definir y encontrar patrones lingüísticos de manera más estructurada que con expresiones regulares, aprovechando el análisis morfosintáctico del lenguaje natural.
<br><br>
<b>4.Aplicabilidad en tareas reales:</b><br>
Este enfoque puede aplicarse para construir sistemas de búsqueda de información en textos largos (por ejemplo, detectar personajes, lugares o temas clave en novelas, noticias o documentos jurídicos), siempre con el beneficio del análisis contextual.
<br><br>
La actividad evidenció que spaCy es una herramienta suficiente y eficiente para la detección de patrones lingüísticos y la extracción de oraciones completas con contexto, gracias a su capacidad de análisis morfosintáctico y al uso del Matcher, lo que permite un procesamiento del texto preciso y flexible.

# **Clasificación de Sentimientos en Críticas Cinematográficas.**
<div align="justify">
Para esta práctica, realizaremos un análisis de sentimientos sobre reseñas de películas. El objetivo es clasificar cada reseña como positiva o negativa, aplicando técnicas de preprocesamiento de texto antes de entrenar o evaluar un modelo de clasificación.

**Creamos el documento desde el archivo filmreviews_sentiment.tsv**

<div align="justify">
Este es un comando de shell que se ejecuta directamente desde el notebook gracias al prefijo !. En lugar de ser interpretado por Python, se envía al sistema operativo como si lo escribieramos en la terminal de Linux que corre detrás de Google Colab.
</div>

In [25]:
!test '{IN_COLAB}' = 'True' && wget  https://raw.githubusercontent.com/YesidCastelblanco/Fundamentos_NLP/refs/heads/main/Unidad1/filmreviews_sentiment.tsv

--2025-08-16 15:42:14--  https://raw.githubusercontent.com/YesidCastelblanco/Fundamentos_NLP/refs/heads/main/Unidad1/filmreviews_sentiment.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 650 [text/plain]
Saving to: ‘filmreviews_sentiment.tsv.5’

filmreviews_sentime 100%[===================>]     650  --.-KB/s    in 0s      

2025-08-16 15:42:14 (45.4 MB/s) - ‘filmreviews_sentiment.tsv.5’ saved [650/650]



**Empecemos por cargar el dataset:**

<div align="justify">
Cargamos el dataset de reseñas de películas desde un archivo .tsv usando pandas, creando un DataFrame con las columnas label (sentimiento) y review (texto de la reseña). Luego, reviews.head() muestra las primeras cinco filas para inspeccionar rápidamente la estructura y verificar que los datos se cargaron correctamente.

In [26]:
reviews = pd.read_csv('./filmreviews_sentiment.tsv', sep='\t')
reviews.head()

,review,label
0,"La trama fue predecible y aburrida, esperaba m...",neg
1,"Una obra maestra, la actuación principal fue i...",pos
2,Los efectos especiales eran mediocres y mal ej...,neg
3,Una historia conmovedora que me hizo llorar de...,pos
4,El guion tenía huecos enormes y personajes poc...,neg


**Limpieza de datos**

---



<div align="justify">
Elimina valores faltantes (NaN) en el DataFrame y limpia los textos de las reseñas eliminando espacios en blanco al inicio y final de cada texto. También identifica y elimina filas que quedaron vacías tras el proceso, asegurando que todas las reseñas contengan contenido válido para el análisis.

In [11]:
reviews.dropna(inplace=True)
reviews.review = reviews.review.apply(lambda r: r.strip())
blanks = reviews[reviews.review == ''].index
reviews.drop(blanks, inplace=True)
reviews[reviews.review == ''].index

Index([], dtype='int64')

In [13]:
reviews.label.value_counts()

,count
label,
neg,5
pos,5


Tenemos un dataset balanceado de casi mil ejemplares por cada clase.

Para hacer las cosas simples, vamos a utilizar un VADER para computar el puntaje de positivo o negativo. Este modelo ya viene implementado dentro de NLTK.

In [14]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [18]:
sid = SentimentIntensityAnalyzer()
reviews['scores'] = reviews.review.apply(lambda r: sid.polarity_scores(r))
reviews.head()

,review,label,scores
0,"La trama fue predecible y aburrida, esperaba m...",neg,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,"Una obra maestra, la actuación principal fue i...",pos,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,Los efectos especiales eran mediocres y mal ej...,neg,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,Una historia conmovedora que me hizo llorar de...,pos,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
4,El guion tenía huecos enormes y personajes poc...,neg,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


Con estos puntajes ahora podemos convertir el resultado en una etiqueta de predicción:

In [19]:
reviews['compound'] = reviews.scores.apply(lambda s: s['compound'])
reviews['prediction'] = reviews['compound'].apply(lambda c: 'pos' if c > 0 else 'neg')
reviews.head()

,review,label,scores,compound,prediction
0,"La trama fue predecible y aburrida, esperaba m...",neg,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,neg
1,"Una obra maestra, la actuación principal fue i...",pos,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,neg
2,Los efectos especiales eran mediocres y mal ej...,neg,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,neg
3,Una historia conmovedora que me hizo llorar de...,pos,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,neg
4,El guion tenía huecos enormes y personajes poc...,neg,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,neg


Y finalmente computar unas cuantas métricas de calidad del modelo:

In [20]:
y_true = reviews.label.values
y_pred = reviews.prediction.values

acc = accuracy_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)
cr = classification_report(y_true, y_pred)


print(f"Accuracy:\n{acc}\n")
print(f"Classification Report:\n{cr}")
print(f"Confusion Matrix:\n{cm}")

Accuracy:
0.5

Classification Report:
              precision    recall  f1-score   support

         neg       0.50      1.00      0.67         5
         pos       0.00      0.00      0.00         5

    accuracy                           0.50        10
   macro avg       0.25      0.50      0.33        10
weighted avg       0.25      0.50      0.33        10

Confusion Matrix:
[[5 0]
 [5 0]]
